In [1]:
# imports
import pandas as pd
import sys
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

In [2]:
# allows animations to be opened in interactive window
%matplotlib tk

In [16]:
# read in files
preds = pd.read_pickle('/Users/Rohil/Documents/iGEM/yemen/y_df_for_feature_selection_new.pkl')
true = pd.read_csv('/Users/Rohil/Documents/iGEM/yemen/cholera_epi_data/yemen_cholera_case_data_differenced.csv')

In [19]:
true.date = pd.to_datetime(true.date, format = '%d-%m-%y')

In [22]:
true = true.set_index('date')

In [53]:
gov_pop_area_data = pd.read_excel('/Users/Rohil/Documents/iGEM/yemen/gov_area_pop_data.xlsx')

In [54]:
gov_pop_area_data = gov_pop_area_data[gov_pop_area_data.iso != 'YE-HD']

In [55]:
for index, row in gov_pop_area_data[['iso', 'population']].iterrows():
    true[row.iso] = (true[row.iso] * 10000) / row.population

In [4]:
pred_crosstab_dict = {}
for col in ['week_1_to_2_cases', 'week_2_to_4_cases', 'week_4_to_6_cases', 'week_6_to_8_cases']:
    
    pred_crosstab_dict[col] = preds[['gov_iso', 'date', col]].pivot_table(index = 'date', columns = 'gov_iso', values = col)

In [25]:
week_12_case_crosstab = pred_crosstab_dict['week_1_to_2_cases']
week_24_case_crosstab = pred_crosstab_dict['week_2_to_4_cases']
week_46_case_crosstab = pred_crosstab_dict['week_4_to_6_cases']
week_68_case_crosstab = pred_crosstab_dict['week_6_to_8_cases']

In [26]:
week_12_case_crosstab.columns

Index(['YE-AB', 'YE-AD', 'YE-AM', 'YE-BA', 'YE-DA', 'YE-DH', 'YE-HD-AL',
       'YE-HJ', 'YE-HU', 'YE-IB', 'YE-JA', 'YE-LA', 'YE-MA', 'YE-MR', 'YE-MW',
       'YE-RA', 'YE-SA', 'YE-SD', 'YE-SH', 'YE-SN', 'YE-TA'],
      dtype='object', name='gov_iso')

In [28]:
true.columns

Index(['YE-AB', 'YE-AD', 'YE-AM', 'YE-BA', 'YE-DA', 'YE-DH', 'YE-HD-AL',
       'YE-HJ', 'YE-HU', 'YE-IB', 'YE-JA', 'YE-LA', 'YE-MA', 'YE-MR', 'YE-MW',
       'YE-RA', 'YE-SA', 'YE-SD', 'YE-SH', 'YE-SN', 'YE-TA'],
      dtype='object')

In [56]:
ylim_df = true.max(axis=0)

In [57]:
y_12_line_dict = {}
y_24_line_dict = {}
y_46_line_dict = {}
y_68_line_dict = {}
true_line_dict = {}

In [58]:
# function that updates plot for each day

def update(i):
    
    # Update the line and the axes (with a new xlabel). Return a tuple of
    # "artists" that have to be redrawn for this frame.
    true_date =  pd.date_range(true.index[0], pred_crosstab_dict['week_1_to_2_cases'].index[i])
    
    dates12 = pd.date_range(pred_crosstab_dict['week_1_to_2_cases'].index[i] + pd.to_timedelta(0, 'W') + pd.to_timedelta(1, 'D'), pred_crosstab_dict['week_1_to_2_cases'].index[i] + pd.to_timedelta(2, 'W'), freq = "D")
    dates24 = pd.date_range(pred_crosstab_dict['week_2_to_4_cases'].index[i] + pd.to_timedelta(2, 'W') + pd.to_timedelta(1, 'D'), pred_crosstab_dict['week_2_to_4_cases'].index[i] + pd.to_timedelta(4, 'W'), freq = "D")
    dates46 = pd.date_range(pred_crosstab_dict['week_4_to_6_cases'].index[i] + pd.to_timedelta(4, 'W') + pd.to_timedelta(1, 'D'), pred_crosstab_dict['week_4_to_6_cases'].index[i] + pd.to_timedelta(6, 'W'), freq = "D")
    dates68 = pd.date_range(pred_crosstab_dict['week_6_to_8_cases'].index[i] + pd.to_timedelta(6, 'W') + pd.to_timedelta(1, 'D'), pred_crosstab_dict['week_6_to_8_cases'].index[i] + pd.to_timedelta(8, 'W'), freq = "D")
    
    for e in range(0,21):
        
        gov = govs[e]
        
        true_val = true.loc[true_date][gov]
        true_line_dict[gov].set_data(true_date, true_val)
        
        vals12 = np.repeat(pred_crosstab_dict['week_1_to_2_cases'][gov].iloc[i] / 14, 14)
        y_12_line_dict[gov].set_data(dates12, vals12)
        
        vals24 = np.repeat(pred_crosstab_dict['week_2_to_4_cases'][gov].iloc[i] / 14, 14)
        y_24_line_dict[gov].set_data(dates24, vals24)

        vals46 = np.repeat(pred_crosstab_dict['week_4_to_6_cases'][gov].iloc[i] / 14, 14)
        y_46_line_dict[gov].set_data(dates46, vals46)

        vals68 = np.repeat(pred_crosstab_dict['week_6_to_8_cases'][gov].iloc[i] / 14, 14)
        y_68_line_dict[gov].set_data(dates68, vals68)
        
    
    label = 'day {0}'.format(i)
    
    ax[-1].set_xlabel(label)
    
    return (ax)

In [66]:
fig, ax = plt.subplots(21,1,figsize = (6,17), sharex=True, sharey = False)

govs = true.columns

for i in range(0,21):

    ax[i].set_xlim(true.index.min(), true.index.max())
    ax[i].set_ylim(0,10)#ylim_df[govs[i]])
    ax[i].legend().set_visible(False)
    ax[i].set_ylabel(govs[i])
    ax[i].yaxis.set_label_position('right')
    ax[i].spines['right'].set_visible(False)
    ax[i].spines['top'].set_visible(False)
    ax[i].spines['bottom'].set_visible(True)


true_date_start =  pd.date_range(true.index[0], pred_crosstab_dict['week_1_to_2_cases'].index[i])

dates12_start = pd.date_range(pred_crosstab_dict['week_1_to_2_cases'].index[0] + pd.to_timedelta(0, 'W') + pd.to_timedelta(1, 'D'), pred_crosstab_dict['week_1_to_2_cases'].index[0] + pd.to_timedelta(2, 'W'), freq = "D")
dates24_start = pd.date_range(pred_crosstab_dict['week_2_to_4_cases'].index[0] + pd.to_timedelta(2, 'W') + pd.to_timedelta(1, 'D'), pred_crosstab_dict['week_2_to_4_cases'].index[0] + pd.to_timedelta(4, 'W'), freq = "D")
dates46_start = pd.date_range(pred_crosstab_dict['week_4_to_6_cases'].index[0] + pd.to_timedelta(4, 'W') + pd.to_timedelta(1, 'D'), pred_crosstab_dict['week_4_to_6_cases'].index[0] + pd.to_timedelta(6, 'W'), freq = "D")
dates68_start = pd.date_range(pred_crosstab_dict['week_6_to_8_cases'].index[0] + pd.to_timedelta(6, 'W') + pd.to_timedelta(1, 'D'), pred_crosstab_dict['week_6_to_8_cases'].index[0] + pd.to_timedelta(8, 'W'), freq = "D")

for e in range(0,21):

    gov = govs[e]
    
    true_val_start = true.loc[true_date_start][gov]
    true_line_dict[gov], = ax[e].plot(true_date_start, true_val_start, color = 'red')

    vals12_start = np.repeat(pred_crosstab_dict['week_1_to_2_cases'][gov].iloc[0] / 14, 14)
    y_12_line_dict[gov], = ax[e].plot(dates12_start, vals12_start, linestyle= '-.', color = 'seagreen')

    vals24_start = np.repeat(pred_crosstab_dict['week_2_to_4_cases'][gov].iloc[0] / 14, 14)
    y_24_line_dict[gov], = ax[e].plot(dates24_start, vals24_start, linestyle= '-.', color = 'blue')

    vals46_start = np.repeat(pred_crosstab_dict['week_4_to_6_cases'][gov].iloc[0] / 14, 14)
    y_46_line_dict[gov], = ax[e].plot(dates46_start, vals46_start, linestyle= '-.', color = 'plum')

    vals68_start = np.repeat(pred_crosstab_dict['week_6_to_8_cases'][gov].iloc[0] / 14, 14)
    y_68_line_dict[gov], = ax[e].plot(dates68_start, vals68_start, linestyle= '-.', color = 'magenta')    
    
anim = FuncAnimation(fig, update, frames=np.arange(0, 200), interval=10)

anim.save('/Users/Rohil/Documents/iGEM/yemen/gif_with_true_vals.gif', dpi=100, writer='imagemagick')

No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
MovieWriter i

In [13]:
fig, ax = plt.subplots()
fig.set_tight_layout(True)

# Query the figure's on-screen size and DPI. Note that when saving the figure to
# a file, we need to provide a DPI for that separately.
print('fig size: {0} DPI, size in inches {1}'.format(
    fig.get_dpi(), fig.get_size_inches()))

ax.set_xlim(pd.Timestamp('2017-05-23'), pd.Timestamp('2018-02-18'))
ax.set_ylim(0, 50)
line, = ax.plot(pd.date_range(pred_crosstab_dict['week_4_to_6_cases'].index[0] + pd.to_timedelta(4, 'W') + pd.to_timedelta(1, 'D'), pred_crosstab_dict['week_4_to_6_cases'].index[0] + pd.to_timedelta(6, 'W'), freq = "D"), np.repeat(pred_crosstab_dict['week_4_to_6_cases']['YE-AB'].iloc[0] / 14, 14), color = 'g')

anim = FuncAnimation(fig, update, frames=np.arange(0, 200), interval=200)

fig size: 100.0 DPI, size in inches [6.4 4.8]
